In [55]:
#!pip install kss

In [90]:
import pandas as pd
import numpy as np
import kss

import gensim
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

from tensorflow.keras.preprocessing.text import Tokenizer
from collections import Counter

In [198]:
df=pd.read_csv('[최종]new_cluster5_인테리어_13.csv')
df['tokenizing'] = df['tokenizing'].apply(eval)
df

,Unnamed: 0,topic,topic_weight,Topic_Keywords,Content,topic_all_weight,tokenizing
0,0,9,0.2871,"추천, 청정기, 공기, 렌탈, 쪽지, 사용, 제품, 고민, 정수기, 정보",코웨이 렌탈 관련 문의기존에 코웨이에서 공기청정기랑 비데를 이용중이었는데 새로 신혼...,"[(9, 0.2871007), (12, 0.24635571), (11, 0.2346...","[프로, 계시, 받다, 네넵, 코웨이, 신혼집, 사원, 추천, 싶다, 댓글, 더, ..."
1,1,10,0.9937,"테라스, 레몬, 반갑, 인테리어, 가구, 장난감, 배치, 책상, 아이, 인사",장난감수납장 어떤거 쓰세요 장난감 수납할 수 있는 정리함 같은거 보고 있는데 뭐가 ...,"[(10, 0.9937266)]","[세이지, 되어다, 가성, 장도, 보고, 디자인, 추천, 들이다, 이번, 사용, 수..."
2,2,11,0.9270,"인테리어, 사진, 생각, 고민, 가능, 사용, 감사, 디자인, 가구, 공간",리뷰 lg 에어로타워 공기청정기 온풍 퓨리케어 오브제 경훈 lg 퓨리케어 오브제컬렉...,"[(11, 0.9270427), (4, 0.04771458), (9, 0.02474...","[춥다, 끼우다, 일반인, 맨, 추천, 원, 꼽, 그대로, 전원, 필터, 부분, 이..."
3,3,1,0.6421,"생각, 인테리어, 사진, 사람, 느낌, 입주, 신축, 거실, 요즘, 보이",자리차지 별로 안하는 예쁜 공기청정기 없나요공기청정기 물론 성능이 제일이지요 다만 ...,"[(1, 0.6420919), (9, 0.35509148)]","[딕, 네, 요전, 카드, 평형, 유, 보통, 귀엽다, 추천, 별로, 적, 두기, ..."
4,4,5,0.5331,"분위기, 인테리어, 예쁘, 느낌, 이쁘, 거실, 보이, 엄청, 꾸미, 신혼집",놀이방 아이방 공간재배치 정리수납 제가 전직 보육교사였기에 놀이방 아이방 정리수납에...,"[(5, 0.53307056), (10, 0.45849884)]","[수납, 보다, 방, 참고, 아이방, 교사, 배치, 꾸미다, 전직, 오늘, 공간, ..."
...,...,...,...,...,...,...,...
3270,3270,10,0.9617,"테라스, 레몬, 반갑, 인테리어, 가구, 장난감, 배치, 책상, 아이, 인사",새로 가입했어요 반갑습니당 반갑습니다 반갑습니다,"[(10, 0.96166)]","[습, 가입, 니당다, 새롭다, 반갑다]"
3271,3271,10,0.9709,"테라스, 레몬, 반갑, 인테리어, 가구, 장난감, 배치, 책상, 아이, 인사",반갑습니다 레몬테라스 가입했어요 잘부탁드립니다 반갑습니다 반갑습니다,"[(10, 0.9709441)]","[부탁드리다, 테라스, 반갑다, 레몬, 가입]"
3272,3272,10,0.9709,"테라스, 레몬, 반갑, 인테리어, 가구, 장난감, 배치, 책상, 아이, 인사",반갑습니다 드디어 레몬테라스에 왔네요 반갑습니다 반갑습니다아 반갑습니다 안녕하세욤 ...,"[(10, 0.970944)]","[테라스, 습, 반갑다, 니당다, 욤, 안녕하다, 레몬]"
3273,3273,10,0.9437,"테라스, 레몬, 반갑, 인테리어, 가구, 장난감, 배치, 책상, 아이, 인사",반가워요 레몬테라스에 가입했어요 반갑습니다 반가워용 반가워요 반가워요 반갑습니다 반...,"[(10, 0.9436618)]","[테라스, 반, 반갑다, 안녕하다, 레몬, 가입]"


In [56]:
# kss 예시
s = "회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요 다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다 강남역 맛집 토끼정의 외부 모습."
for sent in kss.split_sentences(s):
    print(sent)

회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요
다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다 강남역 맛집 토끼정의 외부 모습.


# 문장분리해보자
- 오래 걸림

In [183]:
sentences=[]
reviewer_idx=[]
sentence=df.Content.values.tolist()

for i in range(len(sentence)):
    for sent in kss.split_sentences(sentence[i]):
        sentences.append(sent)
        reviewer_idx.append(i)
        
sent=pd.DataFrame({'reviewer_id':reviewer_idx, 'review':sentences})
sent

,reviewer_id,review
0,0,코웨이 렌탈 관련 문의기존에 코웨이에서 공기청정기랑 비데를 이용중이었는데 새로 신혼...
1,0,혹시 코웨이 영업사원분들 계시면 쪽지좀 부탁드려용 번이 낫지않을까요
2,0,저희집도 정수기 매트리스 렌탈했는데 각각 프로할인받구 만원에 개월공짜였어요
3,0,코웨이담당자님이신데 친절하셔서 추천드리구싶네용 네넵 댓글 감사합니당 쪽지 남겼습니다
4,0,쪽지 닉네임 눈부신햇살이에요
...,...,...
113338,3274,반갑습니다
113339,3274,어서오세요
113340,3274,안녕하세용
113341,3274,어서오세요


# 분리된 문장을 각 토픽에 할당해보자
- LDA 키워드 + 뭔가 토픽에 등장할 만한 단어 찾아서 넣기..(토픽에 등장할 만한 단어를 찾아서 넣는 방법이 없을까요?)
- 코사인 유사도로도 해봤는데 이상한 단어들만 떠서 별 소용이 없어보임

## LDA 키워드 선정하기
- LDA 키워드 선정 
- topic에 할당될 만한 키워드 추가 

In [184]:
#LDA 키워드
action1= ['신혼부부','랜선집들이','집들이','인테리어','스탈','등신대','분위기','스타일','엔티크','센스','취향','조명','화이트','고급','심플']
action2= ['소음','시공','매트','도배','업체','시공업체','설치', '수리','부분수리']
action3= ['책상','배치','공간','가구','이케아','수납','수납장','정리','빌트인','공간']
action4= ['아치형','감각','셀프인테리어','셀프','조화','비포','도배지']
action5= ['노을','가까워','편의','경치','탁트인','평온','새소리','배경','뷰','숲뷰','자연','텃밭','계절감','마운틴']
action6= ['청정기','현금','년치','렌탈파','프로모션','조건','렌탈','공기','정수기','혜택','로봇청소기','프라이어','오븐','코웨이']

In [185]:
# 사전 만들기
name=[action1, action2, action3, action4, action5, action6]

dic={}
for n in range(1,7):
    dic[f'action_{n}'] = name[n-1]
dic

{'action_1': ['신혼부부',
  '랜선집들이',
  '집들이',
  '인테리어',
  '스탈',
  '등신대',
  '분위기',
  '스타일',
  '엔티크',
  '센스',
  '취향',
  '조명',
  '화이트',
  '고급',
  '심플'],
 'action_2': ['소음', '시공', '매트', '도배', '업체', '시공업체', '설치'],
 'action_3': ['책상', '배치', '공간', '가구', '이케아', '수납', '수납장', '정리', '빌트인', '공간'],
 'action_4': ['아치형', '감각', '셀프인테리어', '셀프', '조화', '비포', '도배지'],
 'action_5': ['노을',
  '가까워',
  '편의',
  '경치',
  '탁트인',
  '평온',
  '새소리',
  '배경',
  '뷰',
  '숲뷰',
  '자연',
  '텃밭',
  '계절감',
  '마운틴'],
 'action_6': ['청정기',
  '현금',
  '년치',
  '렌탈파',
  '프로모션',
  '조건',
  '렌탈',
  '공기',
  '정수기',
  '혜택',
  '로봇청소기',
  '프라이어',
  '오븐',
  '코웨이']}

# 키워드 카운트

In [190]:
sent_list = sent.review.values.ravel().tolist()

def calculate_keyword_frequency(data, keywords, topic_num):
    list_=[]
    for row in data:
        # total 초기화
        total=0
        # 키워드별 빈도수를 저장할 딕셔너리 초기화
        frequencies = {keyword: 0 for keyword in keywords}  
        
        for keyword in keywords:
            if keyword in row:
                frequencies[keyword] += 1
        
        # value값 다 더하기
        for value in frequencies.values():
            total+=value
        
        list_.append(total)

    return pd.DataFrame({topic_num: list_})


In [191]:
#열 이름 정하기
action_1='action_1'
action_2='action_2'
action_3='action_3'
action_4='action_4'
action_5='action_5'
action_6='action_6'

result1 = calculate_keyword_frequency(sent_list, dic['action_1'], action_1)
result2 = calculate_keyword_frequency(sent_list, dic['action_2'], action_2)
result3 = calculate_keyword_frequency(sent_list, dic['action_3'], action_3)
result4 = calculate_keyword_frequency(sent_list, dic['action_4'], action_4)
result5 = calculate_keyword_frequency(sent_list, dic['action_5'], action_5)
result6 = calculate_keyword_frequency(sent_list, dic['action_6'], action_6)

total=pd.concat([sent,result1,result2,result3,result4,result5,result6], axis=1)
total

,reviewer_id,review,action_1,action_2,action_3,action_4,action_5,action_6
0,0,코웨이 렌탈 관련 문의기존에 코웨이에서 공기청정기랑 비데를 이용중이었는데 새로 신혼...,0,1,0,0,0,5
1,0,혹시 코웨이 영업사원분들 계시면 쪽지좀 부탁드려용 번이 낫지않을까요,0,0,0,0,0,1
2,0,저희집도 정수기 매트리스 렌탈했는데 각각 프로할인받구 만원에 개월공짜였어요,0,1,0,0,0,2
3,0,코웨이담당자님이신데 친절하셔서 추천드리구싶네용 네넵 댓글 감사합니당 쪽지 남겼습니다,0,0,0,0,0,1
4,0,쪽지 닉네임 눈부신햇살이에요,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
113338,3274,반갑습니다,0,0,0,0,0,0
113339,3274,어서오세요,0,0,0,0,0,0
113340,3274,안녕하세용,0,0,0,0,0,0
113341,3274,어서오세요,0,0,0,0,0,0


In [208]:
# dictionary점수 모든 열이 0인 값 vs. 0이 아닌값을 분리
zero_rows = total[(total['action_1'] == 0)&(total['action_2'] == 0) & (total['action_3'] == 0)& (total['action_4'] == 0)& (total['action_5'] == 0)& (total['action_6'] == 0)]
non_zero_rows = total[~((total['action_1'] == 0)&(total['action_2'] == 0) & (total['action_3'] == 0)& (total['action_4'] == 0)& (total['action_5'] == 0)& (total['action_6'] == 0))]

max_column = non_zero_rows[['action_1', 'action_2', 'action_3', 'action_4','action_5','action_6']].idxmax(axis=1)
non_zero_rows['final_action_num']=[_[-1:] for _ in max_column]
print(non_zero_rows.final_action_num.value_counts())
display(non_zero_rows)
zero_rows

1    17704
3     8471
5     3063
2     2613
6      950
4      635
Name: final_action_num, dtype: int64


/var/folders/sf/g5ffdfm57wv22zjhcdsjt1rm0000gn/T/ipykernel_54320/3188240910.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_zero_rows['final_action_num']=[_[-1:] for _ in max_column]


,reviewer_id,review,action_1,action_2,action_3,action_4,action_5,action_6,final_action_num
0,0,코웨이 렌탈 관련 문의기존에 코웨이에서 공기청정기랑 비데를 이용중이었는데 새로 신혼...,0,1,0,0,0,5,6
1,0,혹시 코웨이 영업사원분들 계시면 쪽지좀 부탁드려용 번이 낫지않을까요,0,0,0,0,0,1,6
2,0,저희집도 정수기 매트리스 렌탈했는데 각각 프로할인받구 만원에 개월공짜였어요,0,1,0,0,0,2,6
3,0,코웨이담당자님이신데 친절하셔서 추천드리구싶네용 네넵 댓글 감사합니당 쪽지 남겼습니다,0,0,0,0,0,1,6
6,1,장난감수납장 어떤거 쓰세요,0,0,2,0,0,0,3
...,...,...,...,...,...,...,...,...,...
112979,3226,다양한 인테리어 정보 공유해요,1,0,0,0,0,0,1
112981,3226,처음 인사 올리는데요 소통하며 지내요 다양한 인테리어 소품과 집꾸미기 방법 공유하고...,1,0,0,0,0,0,1
113065,3237,센스넘치는 분들의 알찬 정보들 감사하게 읽겠습니다,1,0,0,0,0,0,1
113073,3238,레몬테라스 덕에 인테리어에 자신감이 조금 생길것만 같아요,1,0,0,0,0,0,1


,reviewer_id,review,action_1,action_2,action_3,action_4,action_5,action_6
4,0,쪽지 닉네임 눈부신햇살이에요,0,0,0,0,0,0
5,0,본사 소속 상담팀장 소개해드리고 싶네요 상담 넘 친절하게 잘해주셔서 안해도 그냥 물...,0,0,0,0,0,0
8,1,테이블쪽엔 등도 달려 있어요,0,0,0,0,0,0
11,1,온갖 장난감 다 때려넣어서 아이방 엄청 깔끔해졌어요,0,0,0,0,0,0
15,2,사진은 아이폰으로 촬영되었습니다,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
113338,3274,반갑습니다,0,0,0,0,0,0
113339,3274,어서오세요,0,0,0,0,0,0
113340,3274,안녕하세용,0,0,0,0,0,0
113341,3274,어서오세요,0,0,0,0,0,0


In [214]:
print('걸러진 후 남은 reviewer 수: ',len(non_zero_rows.reviewer_id.unique()), '\n원래 reviewer 수: ', len(df))

걸러진 후 남은 reviewer 수:  3034 
원래 reviewer 수:  3275


# 문장저장

In [216]:
non_zero_rows=non_zero_rows[['reviewer_id', 'review', 'final_action_num']]

# cto 코드에 맞춰서 열이름 부여 (Origin=reviewer_id / review=sentences / final_action_num=topic)
non_zero_rows.columns=['origin', 'sentence','topic']
non_zero_rows.to_csv('[최종]new_cluster5_원문', encoding='utf-8-sig')
non_zero_rows

,origin,sentence,topic
0,0,코웨이 렌탈 관련 문의기존에 코웨이에서 공기청정기랑 비데를 이용중이었는데 새로 신혼...,6
1,0,혹시 코웨이 영업사원분들 계시면 쪽지좀 부탁드려용 번이 낫지않을까요,6
2,0,저희집도 정수기 매트리스 렌탈했는데 각각 프로할인받구 만원에 개월공짜였어요,6
3,0,코웨이담당자님이신데 친절하셔서 추천드리구싶네용 네넵 댓글 감사합니당 쪽지 남겼습니다,6
6,1,장난감수납장 어떤거 쓰세요,3
...,...,...,...
112979,3226,다양한 인테리어 정보 공유해요,1
112981,3226,처음 인사 올리는데요 소통하며 지내요 다양한 인테리어 소품과 집꾸미기 방법 공유하고...,1
113065,3237,센스넘치는 분들의 알찬 정보들 감사하게 읽겠습니다,1
113073,3238,레몬테라스 덕에 인테리어에 자신감이 조금 생길것만 같아요,1


# 문장이 잘 부여되었는지 액션별로 확인해본다면!

In [215]:
final_action_1= non_zero_rows[non_zero_rows['final_action_num']=='1'][['reviewer_id', 'review', 'final_action_num']]
final_action_2= non_zero_rows[non_zero_rows['final_action_num']=='2'][['reviewer_id', 'review', 'final_action_num']]
final_action_3= non_zero_rows[non_zero_rows['final_action_num']=='3'][['reviewer_id', 'review', 'final_action_num']]
final_action_4= non_zero_rows[non_zero_rows['final_action_num']=='4'][['reviewer_id', 'review', 'final_action_num']]
final_action_5= non_zero_rows[non_zero_rows['final_action_num']=='5'][['reviewer_id', 'review', 'final_action_num']]
final_action_6= non_zero_rows[non_zero_rows['final_action_num']=='6'][['reviewer_id', 'review', 'final_action_num']]

# 리뷰 확인
display(final_action_1)# 인테리어
display(final_action_2)# 시공
display(final_action_3)# 가구,공간활용
display(final_action_4)# 셀프인테리어
display(final_action_5)# 경치, 뷰
display(final_action_6)# 렌탈

,reviewer_id,review,final_action_num
18,2,색상은 가지 색상만 있는 줄 알고 맨 왼쪽 아이보리 색상으로 구매한다 했는데 구석에...,1
19,2,아이보리도 예쁘긴 한데 화이트가 살짝 더 모던한 느낌이랄까요 가격은 정가 원에서 진...,1
58,5,인테리어를 하자니 결혼준비가 한창이라서 금전적 여유가 없었고 인테리어 비용마저 계속...,1
60,5,셀프 인테리어 턴키 같은 식으로 진행했다고 해야할거 같네요,1
61,5,올 인테리어 디자인 위주로 본다고 하면 도움이 안될 것 같고 적은 돈으로 극강의 효...,1
...,...,...,...
112979,3226,다양한 인테리어 정보 공유해요,1
112981,3226,처음 인사 올리는데요 소통하며 지내요 다양한 인테리어 소품과 집꾸미기 방법 공유하고...,1
113065,3237,센스넘치는 분들의 알찬 정보들 감사하게 읽겠습니다,1
113073,3238,레몬테라스 덕에 인테리어에 자신감이 조금 생길것만 같아요,1


,reviewer_id,review,final_action_num
57,5,신혼집을 누가 매매부터 시작하냐 하는 주변 어른들의 말도 많았지만 오피스텔에서 자취...,2
59,5,엄마의 친한 지인분 중 인테리어 하시는분이 부분수리를 도와주기로 하셨고 기본 견적 ...,2
73,5,정말 신경쓰이고 너무 힘들죠 큰 트러블 없이 진행하긴 했는데 바닥을 외부 업체에서 ...,2
147,5,베란다에 따수미 텐트도 설치할까 생각중이에용,2
196,6,더위 개많이탐 땀 많은편 근데 소음도 예민한편 선풍기 달달거리는소리 자면서도 들려서...,2
...,...,...,...
112373,3197,원하는 벽지 마루바닥 가전 가구 대신 그만큼 돈이 학학학 거실에 글라스폴딩을 설치한...,2
112374,3197,작은방에도 고양이를 키우느라 캣도어넣은 폴딩도어랑 저기 옷걸이들 앞에도 폴딩도어 설...,2
112798,3221,신혼집자금마련 후기 디딤돌대출 개의 실제 후기를 통해 내게 맞는 업체를 직접 골라보세요,2
112848,3222,잘 층간 소음 스트레스 받는중에 가입인사해요,2


,reviewer_id,review,final_action_num
6,1,장난감수납장 어떤거 쓰세요,3
7,1,장난감 수납할 수 있는 정리함 같은거 보고 있는데 뭐가 좀 깔끔하면서 튼튼한가요 세...,3
9,1,장난감 정리함 겸 책상테이블처럼 사용 가능해요,3
10,1,저희집도 세이지폴꺼 들였는데 수납 넉넉하고 넘 좋다는여 디자인도 넘 예쁘구염 수납장...,3
37,2,결론적으로 말씀드리자면 메인 온풍기처럼 사용하기에는 따뜻한 바람이 약하게 느껴집니다...,3
...,...,...,...
112919,3225,디쟈트 가구에서 구매했어요,3
112942,3225,전망도 너무 좋고 가구도 엔틱하고 정말 깔끔하게 잘 꾸미셨어요,3
112943,3225,정리한다고 넘넘 고생많으셨습니다,3
112944,3225,예뻐용 장식장 엄청 이뻐요 저도 술 장식할 장식장 찾고있었는데 딱 찾던거네요 어디서...,3


,reviewer_id,review,final_action_num
75,5,감각좋으시다,4
171,5,와 진짜 최고네요 아치형 문도 색상까지 넘 이쁘구요,4
179,5,감각있으시네요,4
231,8,전체적으로 보니 감각있으신분이신거 같아요,4
316,11,감각이있으신것같네요,4
...,...,...,...
112609,3203,이뻐요 아치형 중문 아치형 중문이 정말 예쁘지요,4
112633,3204,저도 깔끔한 집에서 살고 싶네요 살 아이로 현실은 치워도치워도 와 감각있으시네요,4
112799,3221,신혼집자금마련 후기 디딤돌대출 결혼에 대한 계획도 환상도 없이 갑자기 셀프인테리어에...,4
112916,3225,역시 집꾸미는것도 관심과 감각이 동시에 탑제되어야 저렇게 이쁘게 나오나 봅니다 뷰가...,4


,reviewer_id,review,final_action_num
14,2,결국 에어로 타워를 구매하게 되었고 주일정도 써보면서 느낀 솔직한 후기들을 적어볼게...,5
221,8,흰색쇼파 뒤 액자랑 뒷배경이 책이고 소녀가 앉아있는 액자 정보 부탁드릴께요,5
332,12,햇빛 들어오는방 vs 옆집 벽뷰방 햇빛들어오는방이 배정도 더큼,5
1924,119,집이 좁아진것같은 저희냥이도 슬라이드 사용안하고 투명해먹도 아직 무서운지 안가더라고...,5
2587,140,뷰좋은곳에서 캣휠타서요 캣휠이 거실에 오브제처럼 있는 거 같아요,5
...,...,...,...
112535,3202,도심주택이라 경치랄게 없어요,5
112714,3211,너무 정감있고 좋아보여요 베란다도이쁘구 뷰도좋네요,5
112921,3225,와 창밖 뷰가 정말 좋네요,5
112935,3225,집 뷰가 너무 예쁘네요,5


,reviewer_id,review,final_action_num
0,0,코웨이 렌탈 관련 문의기존에 코웨이에서 공기청정기랑 비데를 이용중이었는데 새로 신혼...,6
1,0,혹시 코웨이 영업사원분들 계시면 쪽지좀 부탁드려용 번이 낫지않을까요,6
2,0,저희집도 정수기 매트리스 렌탈했는데 각각 프로할인받구 만원에 개월공짜였어요,6
3,0,코웨이담당자님이신데 친절하셔서 추천드리구싶네용 네넵 댓글 감사합니당 쪽지 남겼습니다,6
12,2,리뷰 lg 에어로타워 공기청정기 온풍 퓨리케어 오브제 경훈 lg 퓨리케어 오브제컬렉...,6
...,...,...,...
110153,3168,예랑이가 금융권에 있어 매매 대출 받았고 상환기간 없이 퇴직 전 까지만 갚으면 되는...,6
110172,3168,한샘에서 샀어요 현금으로 하면 좀 할인 해 주더라구요,6
110308,3170,저도 티비 큰거 별로 안좋아하는데 눈아파서 사람들은 무조건 큰거라고 하더라구요,6
111302,3181,입주하신거 넘넘 축하드려용 집 너무 좋아보이네요 축하드려요 입주 축하드려요 당분간은...,6
